In [ ]:
#Importing the libraries
import numpy as np
import tensorrec as rec
import pandas as pd
import tensorflow as tf
from scipy import sparse
import random as ran
import tensorflow_datasets as tf_df
#https://towardsdatascience.com/getting-started-with-recommender-systems-and-tensorrec-8f50a9943eef 
#use this link


In [ ]:
#Importing the datasets
dataset1 = pd.read_csv('rating.csv')
dataset2 = pd.read_csv('anime.csv')

In [ ]:
#to merge the do dataframe, we will do a left join between ratings dataset and anims dataset
#we are doing a left join as multiple rows in the anime table can be joined to the rows in rating table

dataset = pd.merge(dataset1,
                  dataset2,
                  left_on = 'anime_id',
                  right_on = 'anime_id',
                  how = 'left')

In [ ]:
dataset1

In [ ]:
dataset2

In [ ]:
dataset

In [ ]:
#checking if the merge produced a cartesian join, because that would indicate something got messed up
print("rating dimensions: {}".format(dataset1.shape))
print("anime dimensions: {}".format(dataset2.shape))
print("final table after left join dimensions: {}".format(dataset.shape))
#since the number of rows are equal to the number os rows in the left table, there is no catesian join and the left join is working perfectly ok!!!


In [ ]:
#splittting the dataset in test and train
#ran.shuffle(dataset)
cutoff = int(.75 * len(dataset))
train_dataset = dataset[:cutoff]
test_dataset = dataset[cutoff:]

In [ ]:
train_dataset.shape

In [ ]:
test_dataset.shape

In [ ]:
user_features = train_dataset.iloc[:, 1:2].values

In [ ]:
item_features = train_dataset.iloc[:, 3:7].values

In [ ]:
ratings = train_dataset.iloc [:, -1].values


In [ ]:

user_features = tf.convert_to_tensor(user_features, dtype=tf.float64)
print(type(user_features))

In [ ]:

item_features = item_features[~pd.isnull(item_features)] #x = x[~numpy.isnan(x)]
item_features = tf.convert_to_tensor(item_features, dtype=tf.string)
print(type(item_features))

In [ ]:
#interactions = tf.convert_to_tensor(interactions, dtype=tf.float32)
#print(type(interactions))

In [ ]:
def interactions_list_to_sparse_matrix(interactions):
    user_features, item_features, ratings, _ = zip(*interactions)
    return sparse.coo_matrix((ratings, (user_features, item_features)), 
                             shape = (n_users, n_iterms))

#create sparse matrices of interaction data
sparse_train_ratings = interactions_list_to_sparse_matrix(train_dataset)
sparse_test_ratings = interactions_list_to_sparse_matrix(test_dataset)


In [ ]:
# Construct indicator features for users and items
user_indicator_features = sparse.identity(n_users)
item_indicator_features = sparse.identity(n_items)

In [ ]:
#Building the model with default parameters

model = rec.TensorRec(n_components = 5)

In [ ]:
# Fit the model for 5 epochs
model.fit(interactions = sparse_train_ratings, 
          user_features=user_indicator_features,
          item_features=item_indicator_features
          , epochs=5, verbose=True)


In [ ]:
# Predict scores and ranks for all users and all items
predictions = model.predict(user_features=user_features,
                            item_features=item_features)
predicted_ranks = model.predict_rank(user_features=user_features,
                                     item_features=item_features)

In [ ]:

# Calculate and print the recall at 10
r_at_k = rec.eval.recall_at_k(predicted_ranks, interactions, k=10)
print(np.mean(r_at_k))